<a href="https://colab.research.google.com/github/gvogiatzis/CS4740/blob/main/CS4740_Lab_Week_05b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploring the capabilities of Recurrent Neural Networks

In this lab we will carry out a few experiments with Recurrent Neural Networks in order to understand the way these models operate and also to see some interesting applications. RNNs are Deep Learning models that are particularly suitable for sequential data such as time series, text, speech, audio signals etc.

We can think of an RNN as a *machine* that is fed a sequence of data-points. After each data-point is entered, the machine performs some calculations based on the information received, and it  modifies its inner state appropriately. It then produces some output before moving to the next data-point in the sequence. 

We can use RNNs in a multitude of ways:

* We can use the last RNN output after a sequence has been read, to make some inference about the type of sequence. E.g. classifying a bit of text into one of a number of categories.
* We can use an RNN to encode an input sequence, and pass a code to a second RNN that decodes that into an output sequence. This can form the basis of a sequence-to-sequence mapping, e.g. translation of english to french text
* We can use an RNN as a next-step predictor. E.g. predicting the next day's stock market price from a s

In [2]:
import re
import csv
from textblob import Word
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from more_itertools import sliced
from torch.utils.tensorboard import SummaryWriter

from random import randint

%load_ext tensorboard

In [3]:
class CounterNet(nn.Module):
    def __init__(self, embed_size=10, hidden_dim=16):
        super(CounterNet, self).__init__()
        self.embedding = nn.Embedding(2, embed_size)
        self.lstm = nn.GRU(input_size=embed_size,
                            hidden_size=hidden_dim,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, batch_size=1):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x).squeeze(dim=2)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size=100
num_of_epochs = 5000
net = CounterNet().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.MSELoss()
for e in range(num_of_epochs):
    # seq_length = randint(10,50)
    x = torch.randint(2, (batch_size,50), device=device)
    t = x.cumsum(dim=1,dtype=torch.float32)
    y = net(x)
    L = loss(y,t)

    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    print(f"\rEpoch: {e}/{num_of_epochs} \tL={L}", end="")

Epoch: 4999/5000 	L=0.12704047560691833

In [5]:
itoc=list('0123456789+=. ')
ctoi = {c:i for i,c in enumerate(itoc)}

def create_strings(a,b):
    c=a+b
    a_s=str(a)
    b_s=str(b)
    c_s=str(c)
    input = f'{a_s}+{b_s}={c_s}'
    output = f'{c_s}.'
    input = f'{a_s}+{b_s}'
    output=' ' * len(input)
    input += f'={c_s}'
    output+= f'{c_s}.'

    
    return input, output

def create_addition_samples(num_samples,ndigits):
    x,t,pairs=[],[],[]
    for n in range(num_samples):
        a = randint(0,10**ndigits-1)
        b = randint(0,10**ndigits-1)
        input, output = create_strings(a,b)
        input = input.ljust(3*ndigits+3, ' ')
        output = output.ljust(3*ndigits+3, ' ')
        x.append([ctoi[c] for c in input])
        t.append([ctoi[c] for c in output])
        pairs.append((a,b))
    return x,t,pairs

In [6]:
class AdderNet(nn.Module):
    def __init__(self, charset=13, embed_size=15, hidden_dim=1024):
        super(AdderNet, self).__init__()
        self.embedding = nn.Embedding(charset, embed_size)
        self.rnn = nn.GRU(input_size=embed_size,
                            hidden_size=hidden_dim,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, charset)
        self.h = None

    def forward(self, x, batch_size=1, keep_memory=False):
        x = self.embedding(x)
        if keep_memory:
            x, self.h = self.rnn(x,self.h)
        else:
            x, self.h = self.rnn(x)
        x = self.fc(x)
        return x

def eval_adder(net, input_str):
    itoc=list('0123456789+=. ')
    ctoi = {c:i for i,c in enumerate(itoc)}
    input = torch.tensor([[ctoi[c] for c in input_str]], device = device)
    x = net(input).argmax(dim=2)
    x = x[0,-1].view(1,-1)
    output = [x.item()]
    total_len=0
    while x.item() != ctoi['.'] and total_len<10:
        x = net(x, keep_memory=True).argmax(dim=2)
        output.append(x.item())
        total_len+=1
    return "".join(itoc[i] for i in output)

In [7]:
training_data=set()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=100
num_of_epochs = 5000
ndigits=3
net = AdderNet(charset = len(itoc)).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()
for e in range(num_of_epochs):
    x,t,pairs = create_addition_samples(batch_size,ndigits=ndigits)
    training_data =training_data.union(set(pairs))
    x = torch.tensor(x,device=device)
    t = torch.tensor(t,device=device)
    y = net(x)
    L = loss(y.view(-1,y.shape[-1]),t.view(-1))

    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    print(f"\rEpoch: {e}/{num_of_epochs} \tL={L}", end="")

Epoch: 4999/5000 	L=0.018413158133625984

In [8]:
all_pairs = set((a,b) for a in range(1000) for b in range(1000))
unseen_data=list(all_pairs.difference(training_data))

In [9]:
errors=0
for i,(a,b) in enumerate(unseen_data):
    input, output = create_strings(a,b)
    # net_output = eval(eval_adder(net, input[:input.find('=')+1]))
    try:
        target = eval(input[:input.find('=')])
        output = eval(eval_adder(net, input[:input.find('=')+1]))
        if target != output:
            # print(input, output,)
            print(f"\r {i}/{len(unseen_data)} {errors/i*100}",end="")
            errors+=1
    except:
        errors+=1

print(f"Error ={errors/len(unseen_data)*100}%")

 606620/606628 6.8260195839240385Error =6.826094410412971%


In [10]:
eval_adder(net, "555+234=")

'789.'